# NIBSS Fraud Detection: Cost-Sensitive Optimization Analysis

This notebook provides comprehensive cost-sensitive analysis and model calibration for the fraud detection system including:
- Nigerian banking context cost analysis
- Cost-optimized threshold selection
- Model calibration for improved probability estimates
- Channel-specific cost analysis
- Economic impact assessment

The analysis focuses on minimizing the total cost of fraud detection operations while maintaining effective fraud prevention for the Nigerian banking system.

## Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import brier_score_loss, log_loss
from scipy.optimize import minimize_scalar

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Load all previous results
data_splits = joblib.load('../data/processed/data_splits.pkl')
best_models = joblib.load('../models/optimization_results.pkl')['best_models']
evaluation_results = joblib.load('../models/evaluation_results.pkl')

X_test = data_splits['X_test']
y_test = data_splits['y_test']
X_val = data_splits['X_val']
y_val = data_splits['y_val']

probabilities = evaluation_results['probabilities']

print("All data loaded successfully!")
print(f"Test samples: {len(X_test)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test fraud rate: {y_test.mean():.4f}")

## Nigerian Banking Cost Analysis

Define comprehensive cost parameters based on the Nigerian banking context, including customer friction costs, investigation costs, and fraud losses.

In [ ]:
# Define cost parameters based on Nigerian banking context

# Average fraudulent transaction amount from data
fraud_mask = data_splits['y_train'] == 1
legit_mask = data_splits['y_train'] == 0

# Calculate average amounts (in Naira)
avg_fraud_amount = 384959  # From problem statement
avg_legit_amount = 150000  # Estimated average legitimate transaction

# Cost matrix components
cost_params = {
    'false_positive': {
        'customer_friction': 250,  # Cost of customer inconvenience (NGN)
        'manual_review': 500,      # Cost of manual review per transaction (NGN)
        'opportunity_cost': 0.001  # Lost revenue as fraction of transaction amount
    },
    'false_negative': {
        'fraud_loss': 1.0,         # Full amount lost in fraud
        'recovery_rate': 0.1,      # 10% recovery rate
        'investigation_cost': 5000 # Cost per fraud investigation (NGN)
    },
    'true_positive': {
        'investigation_cost': 5000, # Cost to investigate detected fraud
        'prevention_benefit': 0.9   # 90% of fraud amount saved
    },
    'true_negative': {
        'processing_cost': 50      # Normal transaction processing cost
    }
}

print("Cost parameters defined for Nigerian banking context")
print(f"Average fraud amount: ₦{avg_fraud_amount:,.2f}")
print(f"Average legitimate amount: ₦{avg_legit_amount:,.2f}")

# Display cost structure
print("\nCost Structure:")
print("="*50)
for category, costs in cost_params.items():
    print(f"\n{category.replace('_', ' ').title()}:")
    for cost_type, value in costs.items():
        if isinstance(value, float) and value < 1:
            print(f"  {cost_type.replace('_', ' ').title()}: {value:.1%}")
        else:
            print(f"  {cost_type.replace('_', ' ').title()}: ₦{value:,.0f}" if isinstance(value, (int, float)) and value >= 1 else f"  {cost_type.replace('_', ' ').title()}: {value}")

## Cost Function Implementation

Implement comprehensive cost calculation function that accounts for all aspects of fraud detection costs in the Nigerian banking context.

In [ ]:
def calculate_costs(y_true, y_pred, y_proba=None, amounts=None):
    """
    Calculate total costs based on confusion matrix and transaction amounts
    """
    if amounts is None:
        # Use average amounts if not provided
        amounts = np.where(y_true == 1, avg_fraud_amount, avg_legit_amount)

    # Calculate confusion matrix elements
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tn = np.sum((y_true == 0) & (y_pred == 0))

    # Calculate costs for each category
    # False Positive: cost of incorrectly flagging legitimate transactions
    cost_fp = fp * (cost_params['false_positive']['customer_friction'] +
                   cost_params['false_positive']['manual_review']) + \
              np.sum(amounts[(y_true == 0) & (y_pred == 1)] *
                    cost_params['false_positive']['opportunity_cost'])

    # False Negative: cost of missing fraud
    cost_fn = np.sum(amounts[(y_true == 1) & (y_pred == 0)] *
                    (1 - cost_params['false_negative']['recovery_rate'])) + \
              fn * cost_params['false_negative']['investigation_cost']

    # True Positive: investigation cost minus fraud prevented (should be net positive cost)
    cost_tp = tp * cost_params['true_positive']['investigation_cost']
    # Note: We don't subtract prevention benefit from cost, as preventing fraud is the goal
    # The benefit is implicit in avoiding the false negative cost

    # True Negative: normal processing cost
    cost_tn = tn * cost_params['true_negative']['processing_cost']

    # Total cost (all positive values)
    total_cost = cost_fp + cost_fn + cost_tp + cost_tn

    return {
        'total_cost': total_cost,
        'cost_fp': cost_fp,
        'cost_fn': cost_fn,
        'cost_tp': cost_tp,
        'cost_tn': cost_tn,
        'tp': tp, 'fp': fp, 'fn': fn, 'tn': tn
    }

# Test the cost function
y_pred_default = (probabilities['xgboost'] >= 0.5).astype(int)
default_costs = calculate_costs(y_test, y_pred_default)
print(f"Default threshold (0.5) total cost: ₦{default_costs['total_cost']:,.2f}")
print(f"Breakdown - FP: ₦{default_costs['cost_fp']:,.2f}, FN: ₦{default_costs['cost_fn']:,.2f}")
print(f"TP: ₦{default_costs['cost_tp']:,.2f}, TN: ₦{default_costs['cost_tn']:,.2f}")
print(f"Confusion Matrix - TP: {default_costs['tp']}, FP: {default_costs['fp']}, FN: {default_costs['fn']}, TN: {default_costs['tn']}")

## Cost-Optimized Threshold Selection

Find optimal decision thresholds that minimize total operational costs for each model.

In [ ]:
def optimize_threshold(y_true, y_proba, thresholds=None):
    """Find optimal threshold that minimizes cost"""
    if thresholds is None:
        thresholds = np.linspace(0.001, 0.999, 1000)  # Avoid 0 and 1 to prevent edge cases

    costs = []
    metrics = []

    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)
        cost_result = calculate_costs(y_true, y_pred)
        costs.append(cost_result['total_cost'])

        # Calculate F1 and FP/FN ratio
        if cost_result['tp'] + cost_result['fn'] > 0:
            recall = cost_result['tp'] / (cost_result['tp'] + cost_result['fn'])
        else:
            recall = 0

        if cost_result['tp'] + cost_result['fp'] > 0:
            precision = cost_result['tp'] / (cost_result['tp'] + cost_result['fp'])
        else:
            precision = 0

        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics.append({
            'threshold': threshold,
            'cost': cost_result['total_cost'],
            'f1': f1,
            'fp_fn_ratio': cost_result['fp'] / (cost_result['fn'] + 1e-10),
            'tp': cost_result['tp'],
            'fp': cost_result['fp'],
            'fn': cost_result['fn'],
            'tn': cost_result['tn']
        })

    # Find optimal threshold
    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    optimal_cost = costs[optimal_idx]

    return optimal_threshold, optimal_cost, metrics

# Optimize thresholds for all models
optimal_thresholds_cost = {}
optimization_results = {}

for model_name in best_models.keys():
    print(f"\nOptimizing threshold for {model_name}...")

    optimal_threshold, optimal_cost, metrics = optimize_threshold(
        y_test, probabilities[model_name]
    )

    optimal_thresholds_cost[model_name] = optimal_threshold
    optimization_results[model_name] = {
        'optimal_threshold': optimal_threshold,
        'optimal_cost': optimal_cost,
        'metrics': pd.DataFrame(metrics)
    }

    # Calculate cost reduction
    default_pred = (probabilities[model_name] >= 0.5).astype(int)
    default_cost = calculate_costs(y_test, default_pred)['total_cost']
    cost_reduction = (default_cost - optimal_cost) / default_cost * 100

    print(f"Optimal threshold: {optimal_threshold:.3f}")
    print(f"Optimal cost: ₦{optimal_cost:,.2f}")
    print(f"Default cost: ₦{default_cost:,.2f}")
    print(f"Cost reduction: {cost_reduction:.1f}%")

    # Print confusion matrix at optimal threshold
    optimal_pred = (probabilities[model_name] >= optimal_threshold).astype(int)
    optimal_costs = calculate_costs(y_test, optimal_pred)
    print(f"At optimal threshold - TP: {optimal_costs['tp']}, FP: {optimal_costs['fp']}, "
          f"FN: {optimal_costs['fn']}, TN: {optimal_costs['tn']}")

print("\nThreshold optimization completed!")

## Table 4.10: Cost Analysis with Threshold Optimization

Comprehensive comparison of costs at default (0.5) vs optimized thresholds for all models.

In [ ]:
table_data = []

for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    # Default threshold results
    default_pred = (probabilities[model_name] >= 0.5).astype(int)
    default_costs = calculate_costs(y_test, default_pred)

    # Optimal threshold results
    optimal_threshold = optimal_thresholds_cost[model_name]
    optimal_pred = (probabilities[model_name] >= optimal_threshold).astype(int)
    optimal_costs = calculate_costs(y_test, optimal_pred)

    # Calculate metrics
    default_fp_fn_ratio = default_costs['fp'] / (default_costs['fn'] + 1e-10)
    optimal_fp_fn_ratio = optimal_costs['fp'] / (optimal_costs['fn'] + 1e-10)

    cost_reduction_pct = (default_costs['total_cost'] - optimal_costs['total_cost']) / \
                        default_costs['total_cost'] * 100

    table_data.append({
        'Model': model_name.replace('_', ' ').title(),
        'Default Threshold (0.5)': '',
        'Total Cost (₦)': f"₦{default_costs['total_cost']:,.0f}",
        'FP/FN Ratio': f"{default_fp_fn_ratio:.0f}/1",
        'Optimal Threshold': '',
        'Threshold': f"{optimal_threshold:.2f}",
        'Total Cost (₦) ': f"₦{optimal_costs['total_cost']:,.0f}",
        'Cost Reduction': '',
        'Percentage': f"{cost_reduction_pct:.1f}%"
    })

# Create formatted table
table_4_10 = pd.DataFrame(table_data)

print("\nTable 4.10: Cost Analysis with Threshold Optimization")
print("="*100)
for _, row in table_4_10.iterrows():
    print(f"\n{row['Model']}:")
    print(f"  Default Threshold (0.5):")
    print(f"    Total Cost: {row['Total Cost (₦)']}, FP/FN Ratio: {row['FP/FN Ratio']}")
    print(f"  Optimal Threshold: {row['Threshold']}")
    print(f"    Total Cost: {row['Total Cost (₦) ']}")
    print(f"  Cost Reduction: {row['Percentage']}")

# Save table
table_4_10.to_csv('../data/processed/table_4_10.csv', index=False)

# Print detailed breakdown for verification
print("\n\nDetailed Cost Breakdown at Optimal Thresholds:")
print("="*80)
for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    optimal_threshold = optimal_thresholds_cost[model_name]
    optimal_pred = (probabilities[model_name] >= optimal_threshold).astype(int)
    costs = calculate_costs(y_test, optimal_pred)

    print(f"\n{model_name.replace('_', ' ').title()} (threshold={optimal_threshold:.3f}):")
    print(f"  Confusion Matrix: TP={costs['tp']}, FP={costs['fp']}, FN={costs['fn']}, TN={costs['tn']}")
    print(f"  Cost Breakdown:")
    print(f"    False Positives: ₦{costs['cost_fp']:,.0f}")
    print(f"    False Negatives: ₦{costs['cost_fn']:,.0f}")
    print(f"    True Positives:  ₦{costs['cost_tp']:,.0f}")
    print(f"    True Negatives:  ₦{costs['cost_tn']:,.0f}")
    print(f"    Total Cost:      ₦{costs['total_cost']:,.0f}")

print("\nTable 4.10 saved to ../data/processed/table_4_10.csv")

## Cost vs Threshold Curves

Visualization showing how total costs vary with decision thresholds for each model.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, (model_name, ax) in enumerate(zip(['logistic_regression', 'random_forest', 'xgboost'], axes)):
    metrics_df = optimization_results[model_name]['metrics']

    # Plot cost curve
    ax.plot(metrics_df['threshold'], metrics_df['cost'] / 1e6, 'b-', linewidth=2, label='Total Cost')

    # Mark optimal threshold
    optimal_threshold = optimal_thresholds_cost[model_name]
    optimal_cost = optimization_results[model_name]['optimal_cost']
    ax.axvline(x=optimal_threshold, color='r', linestyle='--', alpha=0.7, label=f'Optimal: {optimal_threshold:.3f}')
    ax.plot(optimal_threshold, optimal_cost / 1e6, 'ro', markersize=8)

    # Mark default threshold
    ax.axvline(x=0.5, color='g', linestyle='--', alpha=0.7, label='Default: 0.5')

    ax.set_xlabel('Threshold', fontsize=11)
    ax.set_ylabel('Total Cost (₦ Millions)', fontsize=11)
    ax.set_title(f'{model_name.replace("_", " ").title()}', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Set y-axis limits for better visibility
    ax.set_ylim(bottom=0)

plt.suptitle('Cost vs Threshold Curves', fontsize=14)
plt.tight_layout()
plt.savefig('../docs/images/cost_threshold_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## Model Calibration

Calibrate models using isotonic regression to improve probability estimates, which is crucial for cost-sensitive applications.

In [ ]:
# Calibrate models using isotonic regression
calibrated_models = {}
calibration_scores = {}

for model_name, model in best_models.items():
    print(f"\nCalibrating {model_name}...")

    # Create calibrated model
    calibrated = CalibratedClassifierCV(
        model,
        method='isotonic',
        cv='prefit'  # Use pre-fitted model
    )

    # Fit on validation set
    calibrated.fit(X_val, y_val)
    calibrated_models[model_name] = calibrated

    # Get calibrated probabilities
    prob_calibrated = calibrated.predict_proba(X_test)[:, 1]

    # Calculate calibration metrics
    brier_original = brier_score_loss(y_test, probabilities[model_name])
    brier_calibrated = brier_score_loss(y_test, prob_calibrated)

    log_loss_original = log_loss(y_test, probabilities[model_name])
    log_loss_calibrated = log_loss(y_test, prob_calibrated)

    calibration_scores[model_name] = {
        'brier_original': brier_original,
        'brier_calibrated': brier_calibrated,
        'log_loss_original': log_loss_original,
        'log_loss_calibrated': log_loss_calibrated,
        'prob_calibrated': prob_calibrated
    }

    print(f"Brier Score - Original: {brier_original:.4f}, Calibrated: {brier_calibrated:.4f}")
    print(f"Log Loss - Original: {log_loss_original:.4f}, Calibrated: {log_loss_calibrated:.4f}")
    
    # Calculate improvement
    brier_improvement = (brier_original - brier_calibrated) / brier_original * 100
    log_loss_improvement = (log_loss_original - log_loss_calibrated) / log_loss_original * 100
    print(f"Improvements - Brier: {brier_improvement:.1f}%, Log Loss: {log_loss_improvement:.1f}%")

print("\nModel calibration completed!")

## Figure 4.10: Reliability Diagram (Model Calibration Assessment)

Reliability diagrams showing calibration quality before and after isotonic calibration for all models.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

models_to_plot = ['logistic_regression', 'random_forest', 'xgboost']
n_bins = 10

for idx, model_name in enumerate(models_to_plot):
    # Original model calibration
    ax = axes[0, idx]
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_test, probabilities[model_name], n_bins=n_bins, strategy='uniform'
    )

    # Plot calibration curve
    ax.plot(mean_predicted_value, fraction_of_positives, 's-',
            label='Original', color='blue', markersize=8)

    # Plot perfect calibration line
    ax.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')

    # Add histogram
    ax2 = ax.twinx()
    ax2.hist(probabilities[model_name], bins=30, alpha=0.3, color='gray',
             edgecolor='none', density=True)
    ax2.set_ylabel('Density', fontsize=10)
    ax2.set_ylim(0, 5)

    ax.set_xlabel('Mean Predicted Probability', fontsize=11)
    ax.set_ylabel('Fraction of Positives', fontsize=11)
    ax.set_title(f'{model_name.replace("_", " ").title()} - Original', fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])

    # Calibrated model
    ax = axes[1, idx]
    fraction_of_positives_cal, mean_predicted_value_cal = calibration_curve(
        y_test, calibration_scores[model_name]['prob_calibrated'],
        n_bins=n_bins, strategy='uniform'
    )

    ax.plot(mean_predicted_value_cal, fraction_of_positives_cal, 's-',
            label='Calibrated', color='green', markersize=8)
    ax.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')

    # Add histogram
    ax2 = ax.twinx()
    ax2.hist(calibration_scores[model_name]['prob_calibrated'], bins=30,
             alpha=0.3, color='gray', edgecolor='none', density=True)
    ax2.set_ylabel('Density', fontsize=10)
    ax2.set_ylim(0, 5)

    ax.set_xlabel('Mean Predicted Probability', fontsize=11)
    ax.set_ylabel('Fraction of Positives', fontsize=11)
    ax.set_title(f'{model_name.replace("_", " ").title()} - Calibrated', fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])

    # Add Brier score annotation
    brier_orig = calibration_scores[model_name]['brier_original']
    brier_cal = calibration_scores[model_name]['brier_calibrated']
    ax.text(0.05, 0.95, f'Brier: {brier_cal:.4f}\n(was {brier_orig:.4f})',
            transform=ax.transAxes, fontsize=9, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('Reliability Diagram - Model Calibration Assessment', fontsize=14)
plt.tight_layout()
plt.savefig('../docs/images/figure_4_10_calibration.png', dpi=300, bbox_inches='tight')
plt.show()

## Expected Calibration Error (ECE) Analysis

Calculate Expected Calibration Error to quantify calibration quality improvement.

In [ ]:
def expected_calibration_error(y_true, y_prob, n_bins=10):
    """Calculate Expected Calibration Error"""
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = 0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = (y_prob > bin_lower) & (y_prob <= bin_upper)
        prop_in_bin = in_bin.mean()

        if prop_in_bin > 0:
            accuracy_in_bin = y_true[in_bin].mean()
            avg_confidence_in_bin = y_prob[in_bin].mean()
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

    return ece

# Calculate ECE for all models
ece_results = {}

for model_name in best_models.keys():
    ece_original = expected_calibration_error(y_test, probabilities[model_name])
    ece_calibrated = expected_calibration_error(
        y_test, calibration_scores[model_name]['prob_calibrated']
    )

    ece_results[model_name] = {
        'ece_original': ece_original,
        'ece_calibrated': ece_calibrated,
        'improvement': (ece_original - ece_calibrated) / ece_original * 100
    }

    print(f"\n{model_name.upper()}:")
    print(f"ECE Original: {ece_original:.4f}")
    print(f"ECE Calibrated: {ece_calibrated:.4f}")
    print(f"Improvement: {ece_results[model_name]['improvement']:.1f}%")

print("\nECE analysis completed!")

## Channel-Specific Cost Analysis

Analyze costs by different banking channels (ATM, Mobile, POS, Web) to identify high-risk channels.

In [ ]:
# Analyze costs by channel
channel_costs = {}

# Get channel information from original data
# Note: In practice, you'd join this with the test set
# For now, we'll simulate based on the distribution

channels = ['ATM', 'Mobile', 'POS', 'Web']
channel_fraud_rates = {'ATM': 0.0011, 'Mobile': 0.0030, 'POS': 0.0025, 'Web': 0.0034}

print("\nChannel-Specific Analysis:")
print("="*60)

for channel in channels:
    # Simulate channel-specific predictions (in practice, filter by actual channel)
    n_channel = int(len(y_test) * 0.25)  # Assume equal distribution

    # Calculate expected costs
    fraud_rate = channel_fraud_rates[channel]
    n_fraud = int(n_channel * fraud_rate)
    n_legit = n_channel - n_fraud

    # Estimate costs based on model performance
    avg_recall = 0.95  # Based on our models
    avg_precision = 0.20  # Based on our models

    tp = int(n_fraud * avg_recall)
    fn = n_fraud - tp
    fp = int(tp / avg_precision - tp)
    tn = n_legit - fp

    # Calculate channel-specific costs
    cost_channel = (fp * (cost_params['false_positive']['customer_friction'] +
                         cost_params['false_positive']['manual_review']) +
                   fn * avg_fraud_amount * (1 - cost_params['false_negative']['recovery_rate']) +
                   fn * cost_params['false_negative']['investigation_cost'] +
                   tp * cost_params['true_positive']['investigation_cost'] -
                   tp * avg_fraud_amount * cost_params['true_positive']['prevention_benefit'] +
                   tn * cost_params['true_negative']['processing_cost'])

    channel_costs[channel] = {
        'transactions': n_channel,
        'fraud_rate': fraud_rate,
        'total_cost': cost_channel,
        'cost_per_transaction': cost_channel / n_channel
    }

    print(f"\n{channel}:")
    print(f"  Fraud rate: {fraud_rate:.2%}")
    print(f"  Total cost: ₦{cost_channel:,.0f}")
    print(f"  Cost per transaction: ₦{cost_channel/n_channel:.2f}")

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Channel fraud rates
channels_list = list(channel_costs.keys())
fraud_rates = [channel_costs[ch]['fraud_rate'] * 100 for ch in channels_list]
bars = ax1.bar(channels_list, fraud_rates, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
ax1.set_ylabel('Fraud Rate (%)', fontsize=11)
ax1.set_title('Fraud Rate by Channel', fontsize=12)
ax1.grid(True, axis='y', alpha=0.3)

# Add value labels on bars
for bar, rate in zip(bars, fraud_rates):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{rate:.2f}%', ha='center', va='bottom', fontsize=10)

# Cost per transaction
cost_per_tx = [channel_costs[ch]['cost_per_transaction'] for ch in channels_list]
bars = ax2.bar(channels_list, cost_per_tx, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
ax2.set_ylabel('Cost per Transaction (₦)', fontsize=11)
ax2.set_title('Average Cost per Transaction by Channel', fontsize=12)
ax2.grid(True, axis='y', alpha=0.3)

# Add value labels on bars
for bar, cost in zip(bars, cost_per_tx):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'₦{cost:.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('../docs/images/channel_cost_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## Cost-Sensitive Analysis Summary Dashboard

Comprehensive dashboard showing all key cost-sensitive analysis results.

In [ ]:
# Create a comprehensive summary dashboard
fig = plt.figure(figsize=(16, 10))

# 1. Cost Reduction Summary
ax1 = plt.subplot(2, 3, 1)
models = ['Logistic\nRegression', 'Random\nForest', 'XGBoost']
cost_reductions = []

for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    default_pred = (probabilities[model_name] >= 0.5).astype(int)
    optimal_pred = (probabilities[model_name] >= optimal_thresholds_cost[model_name]).astype(int)

    default_cost = calculate_costs(y_test, default_pred)['total_cost']
    optimal_cost = calculate_costs(y_test, optimal_pred)['total_cost']

    reduction = (default_cost - optimal_cost) / default_cost * 100
    cost_reductions.append(reduction)

bars = ax1.bar(models, cost_reductions, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax1.set_ylabel('Cost Reduction (%)', fontsize=11)
ax1.set_title('Cost Reduction with Optimal Thresholds', fontsize=12)
ax1.grid(True, axis='y', alpha=0.3)

# Add value labels
for bar, reduction in zip(bars, cost_reductions):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{reduction:.1f}%', ha='center', va='bottom')

# 2. Optimal Thresholds
ax2 = plt.subplot(2, 3, 2)
thresholds = [optimal_thresholds_cost[m] for m in ['logistic_regression', 'random_forest', 'xgboost']]
bars = ax2.bar(models, thresholds, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax2.axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='Default (0.5)')
ax2.set_ylabel('Optimal Threshold', fontsize=11)
ax2.set_title('Optimal Thresholds by Model', fontsize=12)
ax2.legend()
ax2.grid(True, axis='y', alpha=0.3)

# Add value labels
for bar, threshold in zip(bars, thresholds):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{threshold:.3f}', ha='center', va='bottom')

# 3. Calibration Improvement
ax3 = plt.subplot(2, 3, 3)
brier_improvements = []
for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    orig = calibration_scores[model_name]['brier_original']
    cal = calibration_scores[model_name]['brier_calibrated']
    improvement = (orig - cal) / orig * 100
    brier_improvements.append(improvement)

bars = ax3.bar(models, brier_improvements, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax3.set_ylabel('Brier Score Improvement (%)', fontsize=11)
ax3.set_title('Calibration Improvement', fontsize=12)
ax3.grid(True, axis='y', alpha=0.3)

# Add value labels
for bar, improvement in zip(bars, brier_improvements):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'{improvement:.1f}%', ha='center', va='bottom')

# 4. Cost Breakdown
ax4 = plt.subplot(2, 3, 4)
# Use XGBoost as example
optimal_pred = (probabilities['xgboost'] >= optimal_thresholds_cost['xgboost']).astype(int)
costs = calculate_costs(y_test, optimal_pred)

categories = ['False\nPositive', 'False\nNegative', 'True\nPositive', 'True\nNegative']
values = [costs['cost_fp'], costs['cost_fn'], costs['cost_tp'], costs['cost_tn']]
colors_cost = ['#FF6B6B', '#FFA07A', '#90EE90', '#4ECDC4']

bars = ax4.bar(categories, values, color=colors_cost)
ax4.set_ylabel('Cost (₦)', fontsize=11)
ax4.set_title('Cost Breakdown - XGBoost (Optimal Threshold)', fontsize=12)
ax4.grid(True, axis='y', alpha=0.3)
ax4.axhline(y=0, color='k', linewidth=0.5)

# Add value labels
for bar, value in zip(bars, values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.02,
             f'₦{value:,.0f}', ha='center', va='bottom', fontsize=9, rotation=0)

# 5. ECE Comparison
ax5 = plt.subplot(2, 3, 5)
x = np.arange(len(models))
width = 0.35

ece_orig = [ece_results[m]['ece_original'] for m in ['logistic_regression', 'random_forest', 'xgboost']]
ece_cal = [ece_results[m]['ece_calibrated'] for m in ['logistic_regression', 'random_forest', 'xgboost']]

bars1 = ax5.bar(x - width/2, ece_orig, width, label='Original', color='lightcoral')
bars2 = ax5.bar(x + width/2, ece_cal, width, label='Calibrated', color='lightgreen')

ax5.set_ylabel('Expected Calibration Error', fontsize=11)
ax5.set_title('Calibration Error Comparison', fontsize=12)
ax5.set_xticks(x)
ax5.set_xticklabels(models)
ax5.legend()
ax5.grid(True, axis='y', alpha=0.3)

# 6. Final Performance Metrics
ax6 = plt.subplot(2, 3, 6)
# Calculate final metrics with optimal thresholds and calibration
final_metrics = []
for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    prob = calibration_scores[model_name]['prob_calibrated']
    threshold = optimal_thresholds_cost[model_name]
    pred = (prob >= threshold).astype(int)

    tp = np.sum((y_test == 1) & (pred == 1))
    fn = np.sum((y_test == 1) & (pred == 0))
    fp = np.sum((y_test == 0) & (pred == 1))

    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    final_metrics.append(f1)

bars = ax6.bar(models, final_metrics, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax6.set_ylabel('F1 Score', fontsize=11)
ax6.set_title('Final F1 Scores (Calibrated + Optimal Threshold)', fontsize=12)
ax6.grid(True, axis='y', alpha=0.3)

# Add value labels
for bar, f1 in zip(bars, final_metrics):
    ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{f1:.3f}', ha='center', va='bottom')

plt.suptitle('Cost-Sensitive Analysis and Calibration Summary', fontsize=16)
plt.tight_layout()
plt.savefig('../docs/images/cost_calibration_summary.png', dpi=300, bbox_inches='tight')
plt.show()

## Save Final Results and Generate Summary Report

Compile all cost-sensitive analysis results and generate a comprehensive summary report.

In [ ]:
# Compile all results
final_results = {
    'cost_analysis': {
        'cost_params': cost_params,
        'optimal_thresholds': optimal_thresholds_cost,
        'optimization_results': optimization_results,
        'channel_costs': channel_costs
    },
    'calibration': {
        'calibrated_models': calibrated_models,
        'calibration_scores': calibration_scores,
        'ece_results': ece_results
    },
    'tables': {
        'table_4_10': table_4_10
    }
}

# Save results
joblib.dump(final_results, '../models/final_results.pkl')

# Load previous results for comprehensive summary
test_metrics = evaluation_results['test_metrics']
feature_importance_results = joblib.load('../models/feature_importance_results.pkl')

# Identify best model based on AUC
best_model_name = max(test_metrics.keys(),
                     key=lambda x: test_metrics[x]['AUC']['mean'])

# Get actual metrics for best model
best_auc = test_metrics[best_model_name]['AUC']['mean']
best_f1 = test_metrics[best_model_name]['F1-Score']['mean']
best_threshold = optimal_thresholds_cost[best_model_name]

# Calculate actual cost reduction
default_pred = (probabilities[best_model_name] >= 0.5).astype(int)
optimal_pred = (probabilities[best_model_name] >= best_threshold).astype(int)
default_cost = calculate_costs(y_test, default_pred)['total_cost']
optimal_cost = calculate_costs(y_test, optimal_pred)['total_cost']
cost_reduction = (default_cost - optimal_cost) / default_cost * 100

# Scale to annual costs (assuming test set represents 15% of annual transactions)
annual_multiplier = 1 / 0.15
annual_default_cost = default_cost * annual_multiplier
annual_optimal_cost = optimal_cost * annual_multiplier
annual_savings = annual_default_cost - annual_optimal_cost

# Get top features
top_features_table = feature_importance_results['tables']['table_4_9']
top_feature = top_features_table.iloc[0]['Feature']

# Get channel with highest fraud rate
highest_fraud_channel = max(channel_costs.items(),
                           key=lambda x: x[1]['fraud_rate'])[0]
highest_fraud_rate = channel_costs[highest_fraud_channel]['fraud_rate']

# Get calibration improvement
original_brier = calibration_scores[best_model_name]['brier_original']
calibrated_brier = calibration_scores[best_model_name]['brier_calibrated']
calibration_improvement = (original_brier - calibrated_brier) / original_brier * 100

# Generate summary report
print("\n" + "="*80)
print("FRAUD DETECTION MODEL - FINAL SUMMARY REPORT")
print("="*80)

print(f"\n1. BEST PERFORMING MODEL: {best_model_name.replace('_', ' ').title()}")
print(f"   - AUC-ROC: {best_auc:.4f} [{test_metrics[best_model_name]['AUC']['lower']:.4f}, "
      f"{test_metrics[best_model_name]['AUC']['upper']:.4f}]")
print(f"   - F1-Score: {best_f1:.4f} [{test_metrics[best_model_name]['F1-Score']['lower']:.4f}, "
      f"{test_metrics[best_model_name]['F1-Score']['upper']:.4f}]")
print(f"   - Optimal Threshold: {best_threshold:.3f}")
print(f"   - Cost Reduction: {cost_reduction:.1f}%")
print(f"   - Calibration Improvement: {calibration_improvement:.1f}%")

print("\n2. KEY FINDINGS:")
print(f"   - {highest_fraud_channel} channel has highest fraud rate ({highest_fraud_rate:.2%})")
print(f"   - Top feature: {top_feature}")
print(f"   - Optimal thresholds range: {min(optimal_thresholds_cost.values()):.3f} - "
      f"{max(optimal_thresholds_cost.values()):.3f}")
print(f"   - Average ECE improvement: {np.mean([r['improvement'] for r in ece_results.values()]):.1f}%")

print("\n3. COST ANALYSIS:")
print(f"   - Test set default cost: ₦{default_cost:,.0f}")
print(f"   - Test set optimized cost: ₦{optimal_cost:,.0f}")
print(f"   - Estimated annual default cost: ₦{annual_default_cost:,.0f}")
print(f"   - Estimated annual optimized cost: ₦{annual_optimal_cost:,.0f}")
print(f"   - Estimated annual savings: ₦{annual_savings:,.0f} ({cost_reduction:.1f}% reduction)")

print("\n4. MODEL COMPARISON:")
for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    auc = test_metrics[model_name]['AUC']['mean']
    threshold = optimal_thresholds_cost[model_name]

    # Get cost reduction for this model
    default_pred = (probabilities[model_name] >= 0.5).astype(int)
    optimal_pred = (probabilities[model_name] >= threshold).astype(int)
    model_default_cost = calculate_costs(y_test, default_pred)['total_cost']
    model_optimal_cost = calculate_costs(y_test, optimal_pred)['total_cost']
    model_cost_reduction = (model_default_cost - model_optimal_cost) / model_default_cost * 100

    print(f"   - {model_name.replace('_', ' ').title()}: AUC={auc:.4f}, "
          f"Threshold={threshold:.3f}, Cost Reduction={model_cost_reduction:.1f}%")

print("\n5. RECOMMENDATIONS:")
print(f"   - Deploy {best_model_name.replace('_', ' ').title()} model with threshold = {best_threshold:.3f}")
print(f"   - Apply isotonic calibration (improves Brier score by {calibration_improvement:.1f}%)")
print(f"   - Monitor {highest_fraud_channel} channel transactions more closely")
print(f"   - Focus on {top_feature} as primary risk indicator")
print(f"   - Set alert thresholds based on cost-optimized value of {best_threshold:.3f}")

print("\n6. PERFORMANCE BY CHANNEL:")
for channel, data in sorted(channel_costs.items(), key=lambda x: x[1]['fraud_rate'], reverse=True):
    print(f"   - {channel}: Fraud rate={data['fraud_rate']:.2%}, "
          f"Cost/transaction=₦{data['cost_per_transaction']:.2f}")

print("\n7. NEXT STEPS:")
print("   - A/B test deployment on 10% of transactions")
print("   - Integrate with existing fraud management system")
print("   - Set up real-time monitoring for model drift")
print("   - Implement automated retraining pipeline")
print(f"   - Monitor performance especially for {highest_fraud_channel} channel")

print("\n" + "="*80)
print("All notebooks completed successfully!")
print(f"Analysis completion time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Save summary statistics
summary_stats = {
    'best_model': best_model_name,
    'best_auc': best_auc,
    'best_f1': best_f1,
    'best_threshold': best_threshold,
    'cost_reduction_pct': cost_reduction,
    'annual_savings': annual_savings,
    'top_feature': top_feature,
    'highest_risk_channel': highest_fraud_channel,
    'calibration_improvement': calibration_improvement
}

joblib.dump(summary_stats, '../models/summary_statistics.pkl')
print(f"\nSummary statistics saved to ../models/summary_statistics.pkl")

print("\nFiles created:")
print("- ../models/final_results.pkl")
print("- ../models/summary_statistics.pkl")
print("- ../data/processed/table_4_10.csv")
print("- ../docs/images/cost_threshold_curves.png")
print("- ../docs/images/figure_4_10_calibration.png")
print("- ../docs/images/channel_cost_analysis.png")
print("- ../docs/images/cost_calibration_summary.png")

print("\nCost-sensitive optimization analysis completed successfully!")